## Day5-02: 部署 ADK Agent 到 Vertex AI Agent Engine

5 天 AI Agents 强化课程的最后一部分！在前一个 notebook 中你学会了如何使用 A2A 协议让 Agent 互操作。

现在，让我们迈出最后一步：使用 [Vertex AI Agent Engine](https://docs.cloud.google.com/agent-builder/agent-engine/overview) 将 Agent 部署到生产环境。

你将学会：

- 构建生产就绪的 ADK Agent
- 使用 ADK CLI 部署到 **Vertex AI Agent Engine**
- 使用 Python SDK 测试已部署的 Agent
- 在 Google Cloud Console 中监控和管理 Agent
- 了解如何使用 Vertex AI Memory Bank 添加长期记忆
- 成本管理和清理最佳实践

## 精简总结：Agent 部署

### 💡 为什么需要部署？

你构建了一个很棒的 AI Agent。它在你的机器上完美运行。但有一个问题：

> **你的 Agent 没有公开可用！**

- 它只存在于你的 notebook 和开发环境中
- 当你停止 notebook 会话时，它就停止工作
- 你的队友无法访问它
- 你的用户无法与它交互

这正是我们需要部署 Agent 的原因！

### 部署选项对比

| 平台 | 特点 | 适用场景 |
|------|------|----------|
| **Agent Engine** | 全托管，自动扩缩，内置会话管理 | 生产环境首选 |
| **Cloud Run** | 无服务器，最简单 | 演示和中小型负载 |
| **GKE** | 完全控制容器化部署 | 复杂多 Agent 系统 |

### Agent Engine 部署文件结构

```
sample_agent/
├── agent.py                  # Agent 逻辑
├── requirements.txt          # 依赖库
├── .env                      # 配置
└── .agent_engine_config.json # 硬件规格
```

### 关键命令

```bash
# 部署 Agent
adk deploy agent_engine --project=PROJECT_ID --region=REGION sample_agent

# 删除 Agent（避免费用）
agent_engines.delete(resource_name=agent.resource_name, force=True)
```

本次课程直播回放请到 youtube 观看: [youtube](https://www.youtube.com/playlist?list=PLqFaTIg4myu9r7uRoNfbJhHUbLp-1t1YE)

---
## ⚙️ Section 1: 设置环境

### ⚠️ 重要：前提条件

这个 notebook 需要 **Google Cloud Platform (GCP) 账号** 才能将 Agent 部署到 Vertex AI Agent Engine。

**如果你还没有 GCP 账号：**

✅ **步骤 1. 创建免费 Google Cloud 账号** - [在此注册](https://cloud.google.com/free)
- 新用户获得 **$300 免费额度**，有效期 90 天
- 免费试用期间不会收费

✅ **步骤 2. 在你的账号上启用计费** - 即使是免费层也需要
- 需要信用卡进行验证
- 除非你明确升级，否则不会收费
- 如果及时清理资源，这个演示保持在免费层内

✅ **步骤 3. 了解免费层** - 知道包含什么
- 免费部署最多 10 个 Agent
- 查看[免费层详情](https://cloud.google.com/free/docs/free-cloud-features#free-trial)

**💡 快速设置指南：** 观看这个 [3 分钟设置视频](https://youtu.be/-nUAQq_evxc) 获取演示

### 1.1 导入组件

In [ ]:
import os
import random
import time
import vertexai
from vertexai import agent_engines

print("✅ 导入完成")

### 1.2 配置 Google Cloud 凭据

你需要设置 Google Cloud 凭据。有几种方式：

**方式 1：使用 gcloud CLI（推荐用于本地开发）**
```bash
gcloud auth application-default login
```

**方式 2：使用服务账号密钥**
```python
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/service-account-key.json"
```

In [ ]:
# 如果你已经通过 gcloud auth application-default login 登录，
# 这一步可以跳过

# 或者设置服务账号凭据路径
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/your/service-account-key.json"

print("✅ 请确保已配置 Google Cloud 凭据")

### 1.3 设置你的 PROJECT_ID

**重要：** 确保将 `"your-project-id"` 替换为你实际的 Google Cloud 项目 ID。你可以在 [Google Cloud Console](https://console.cloud.google.com/) 中找到你的项目 ID。

In [ ]:
## 设置你的 PROJECT_ID
PROJECT_ID = "your-project-id"  # TODO: 替换为你的项目 ID
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

if PROJECT_ID == "your-project-id" or not PROJECT_ID:
    raise ValueError("⚠️ 请将 'your-project-id' 替换为你实际的 Google Cloud 项目 ID。")

print(f"✅ 项目 ID 设置为: {PROJECT_ID}")

### 1.4 启用 Google Cloud API

对于本教程，你需要在 Google Cloud Console 中启用以下 API：

- Vertex AI API
- Cloud Storage API
- Cloud Logging API
- Cloud Monitoring API
- Cloud Trace API
- Telemetry API

你可以[使用此链接打开 Google Cloud Console](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,storage.googleapis.com,logging.googleapis.com,monitoring.googleapis.com,cloudtrace.googleapis.com,telemetry.googleapis.com) 并按照那里的步骤启用这些 API。

---
## 🏗️ Section 2: 使用 ADK 创建你的 Agent

在部署之前，我们需要一个功能性的 Agent 来托管。我们将构建一个**天气助手**作为示例 Agent。

这个 Agent 针对生产测试进行了优化，配置如下：

- **模型**：使用 `gemini-2.5-flash-lite` 以获得低延迟和成本效益
- **工具**：包含 `get_weather` 函数来演示工具执行
- **人设**：以对话方式响应，证明指令跟随能力

这演示了我们即将打包的基础 ADK 架构：**Agent + Tools + Instructions**。

我们将创建以下文件和目录结构：

```
sample_agent/
├── agent.py                  # 逻辑
├── requirements.txt          # 库
├── .env                      # 配置
└── .agent_engine_config.json # 硬件规格
```

### 2.1 创建 Agent 目录

我们需要一个干净的工作空间来打包 Agent 以进行部署。我们将创建一个名为 `sample_agent` 的目录。

In [ ]:
## 创建简单的 Agent - Agent 的所有代码将放在这个目录中
!mkdir -p sample_agent

print(f"✅ Sample Agent 目录创建完成")

### 2.2 创建 requirements 文件

Agent Engine 为你的 Agent 构建专用环境。为确保它正确运行，我们必须声明依赖项。

In [ ]:
%%writefile sample_agent/requirements.txt

google-adk
opentelemetry-instrumentation-google-genai

### 2.3 创建环境配置

我们需要为 Agent 提供必要的云配置设置。

In [ ]:
%%writefile sample_agent/.env

# https://cloud.google.com/vertex-ai/generative-ai/docs/learn/locations#global-endpoint
GOOGLE_CLOUD_LOCATION="global"

# 设置为 1 使用 Vertex AI，或 0 使用 Google AI Studio
GOOGLE_GENAI_USE_VERTEXAI=1

**配置说明：**

- `GOOGLE_CLOUD_LOCATION="global"` - 使用 `global` 端点进行 Gemini API 调用
- `GOOGLE_GENAI_USE_VERTEXAI=1` - 配置 ADK 使用 Vertex AI 而不是 Google AI Studio

### 2.4 创建 Agent 代码

现在我们将生成 `agent.py` 文件。这个脚本定义了我们的**天气助手**的行为。

Agent 配置：

- 🧠 模型：使用 `gemini-2.5-flash-lite` 以获得低延迟和成本效益
- 🛠️ 工具：访问 `get_weather` 函数来获取数据
- 📝 指令：遵循系统提示来识别城市并以友好的语气回应

In [ ]:
%%writefile sample_agent/agent.py
from google.adk.agents import Agent
import vertexai
import os

vertexai.init(
    project=os.environ["GOOGLE_CLOUD_PROJECT"],
    location=os.environ["GOOGLE_CLOUD_LOCATION"],
)

def get_weather(city: str) -> dict:
    """
    返回给定城市的天气信息。

    这是 Agent 可以在用户询问天气时调用的工具。
    在生产环境中，这会调用真实的天气 API（如 OpenWeatherMap）。
    对于这个演示，我们使用模拟数据。

    Args:
        city: 城市名称（如 "东京"、"纽约"）

    Returns:
        dict: 包含状态和天气报告或错误信息的字典
    """
    # 模拟天气数据库，带有结构化响应
    weather_data = {
        "san francisco": {"status": "success", "report": "旧金山天气晴朗，温度 22°C (72°F)。"},
        "new york": {"status": "success", "report": "纽约多云，温度 18°C (65°F)。"},
        "london": {"status": "success", "report": "伦敦有雨，温度 14°C (58°F)。"},
        "tokyo": {"status": "success", "report": "东京天气晴朗，温度 21°C (70°F)。"},
        "paris": {"status": "success", "report": "巴黎局部多云，温度 20°C (68°F)。"},
        "beijing": {"status": "success", "report": "北京天气晴朗，温度 25°C (77°F)。"},
        "shanghai": {"status": "success", "report": "上海多云，温度 23°C (73°F)。"},
    }

    city_lower = city.lower()
    if city_lower in weather_data:
        return weather_data[city_lower]
    else:
        available_cities = ", ".join([c.title() for c in weather_data.keys()])
        return {
            "status": "error",
            "error_message": f"'{city}' 的天气信息不可用。请尝试: {available_cities}"
        }

root_agent = Agent(
    name="weather_assistant",
    model="gemini-2.5-flash-lite",  # 快速、经济的 Gemini 模型
    description="一个提供城市天气信息的友好天气助手。",
    instruction="""
    你是一个友好的天气助手。当用户询问天气时：

    1. 从他们的问题中识别城市名称
    2. 使用 get_weather 工具获取当前天气信息
    3. 以友好、对话的语气回应
    4. 如果城市不可用，建议可用的城市之一

    在回应中保持友好和简洁。
    """,
    tools=[get_weather]
)

---
## ☁️ Section 3: 部署到 Agent Engine

ADK 支持多种部署平台。在 [ADK 部署文档](https://google.github.io/adk-docs/deploy/) 中了解更多。

你将在这个 notebook 中部署到 [Vertex AI Agent Engine](https://docs.cloud.google.com/agent-builder/agent-engine/overview)。

### 🔷 Vertex AI Agent Engine

- **全托管**服务，专为 AI Agent 设计
- 内置**自动扩缩**和会话管理
- 使用 [Agent Starter Pack](https://github.com/GoogleCloudPlatform/agent-starter-pack) **轻松部署**
- 📚 [部署到 Agent Engine 指南](https://google.github.io/adk-docs/deploy/agent-engine/)

**注意**：为帮助你开始使用运行时，Agent Engine 提供每月免费层，你可以在[文档](https://docs.cloud.google.com/agent-builder/agent-engine/overview#pricing)中了解更多。如果及时清理，本 notebook 中部署的 Agent 应该保持在免费层内。注意如果 Agent 持续运行可能会产生费用。

### 🚢 其他部署选项

- **Cloud Run**：无服务器，最容易上手，适合演示和中小型负载
- **GKE**：完全控制容器化部署，适合复杂多 Agent 系统

### 3.1 创建部署配置

`.agent_engine_config.json` 文件控制部署设置。

In [ ]:
%%writefile sample_agent/.agent_engine_config.json
{
    "min_instances": 0,
    "max_instances": 1,
    "resource_limits": {"cpu": "1", "memory": "1Gi"}
}

**配置说明：**

- `"min_instances": 0` - 不使用时缩减到零（节省成本）
- `"max_instances": 1` - 最多运行 1 个实例（对这个演示足够）
- `"cpu": "1"` - 每个实例 1 个 CPU 核心
- `"memory": "1Gi"` - 每个实例 1 GB 内存

这些设置在为天气 Agent 提供足够资源的同时保持成本最小化。

### 3.2 选择部署区域

Agent Engine 在特定区域可用。我们将为这个演示随机选择一个。

In [ ]:
regions_list = ["europe-west1", "europe-west4", "us-east4", "us-west1"]
deployed_region = random.choice(regions_list)

print(f"✅ 选择的部署区域: {deployed_region}")

**关于区域：**

Agent Engine 在多个区域可用。对于生产环境：

- 选择靠近用户的区域以降低延迟
- 考虑数据驻留要求
- 查看 [Agent Engine 位置文档](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview#locations)

### 3.3 部署 Agent

使用 ADK CLI 将你的 Agent 部署到 Agent Engine。

In [ ]:
!adk deploy agent_engine --project=$PROJECT_ID --region=$deployed_region sample_agent --agent_engine_config_file=sample_agent/.agent_engine_config.json

**刚才发生了什么：**

`adk deploy agent_engine` 命令：

1. 打包你的 Agent 代码（`sample_agent/` 目录）
2. 上传到 Agent Engine
3. 创建容器化部署
4. 输出资源名称，如：`projects/PROJECT_NUMBER/locations/REGION/reasoningEngines/ID`

**注意：** 部署通常需要 2-5 分钟。

---
## 🤖 Section 4: 获取和测试已部署的 Agent

### 4.1 获取已部署的 Agent

使用 CLI 部署后，我们需要获取 Agent 对象以与其交互。

In [ ]:
# 初始化 Vertex AI
vertexai.init(project=PROJECT_ID, location=deployed_region)

# 获取最近部署的 Agent
agents_list = list(agent_engines.list())
if agents_list:
    remote_agent = agents_list[0]  # 获取第一个（最近的）Agent
    client = agent_engines
    print(f"✅ 已连接到已部署的 Agent: {remote_agent.resource_name}")
else:
    print("❌ 未找到 Agent。请先部署。")

**发生了什么：**

这个 cell 获取你已部署的 Agent：

1. 用你的项目和区域初始化 Vertex AI SDK
2. 列出该区域的所有已部署 Agent
3. 获取第一个（最近部署的）
4. 将其存储为 `remote_agent` 以供测试

### 4.2 测试已部署的 Agent

现在让我们向已部署的 Agent 发送查询！

In [ ]:
async for item in remote_agent.async_stream_query(
    message="北京的天气怎么样？",
    user_id="user_42",
):
    print(item)

**发生了什么：**

这个 cell 测试你已部署的 Agent：

1. 发送查询 "北京的天气怎么样？"
2. 流式传输 Agent 的响应

**理解输出：**

你会看到打印多个项目：

1. **函数调用** - Agent 决定调用 `get_weather` 工具
2. **函数响应** - 工具的结果（天气数据）
3. **最终响应** - Agent 的自然语言答案

In [ ]:
# 测试另一个城市
async for item in remote_agent.async_stream_query(
    message="东京今天天气如何？",
    user_id="user_42",
):
    print(item)

---
## 🧠 Section 5: 使用 Vertex AI Memory Bank 的长期记忆

### 什么问题 Memory Bank 解决？

你已部署的 Agent 有**会话记忆** - 它在聊天时记住对话。但一旦会话结束，它会忘记一切。每次新对话都从头开始。

**问题：**

- 用户今天告诉 Agent "我喜欢摄氏度"
- 明天，用户询问天气 → Agent 给出华氏度（忘记了偏好）
- 用户每次都要重复偏好

### 💡 什么是 Vertex AI Memory Bank？

Memory Bank 给你的 Agent **跨会话的长期记忆**：

| 会话记忆 | Memory Bank |
|---------|-------------|
| 单次对话 | 所有对话 |
| 会话结束后忘记 | 永久记住 |
| "我刚才说了什么？" | "我最喜欢的城市是什么？" |

**工作原理：**

1. **对话期间** - Agent 使用记忆工具搜索过去的事实
2. **对话结束后** - Agent Engine 提取关键信息（"用户偏好摄氏度"）
3. **下次会话** - Agent 自动回忆并使用该信息

**示例：**

- **会话 1：** 用户："我偏好摄氏度"
- **会话 2（几天后）：** 用户："东京天气？" → Agent 自动以摄氏度回应 ✨

### 🔧 Memory Bank 和你的部署

你的 Agent Engine 部署**提供** Memory Bank 的基础设施，但默认不启用。

**要使用 Memory Bank：**

1. 在 Agent 代码中添加记忆工具（`PreloadMemoryTool`）
2. 添加回调以将对话保存到 Memory Bank
3. 重新部署你的 Agent

配置后，Memory Bank 自动工作 - 不需要额外的基础设施！

### 📚 了解更多

- **[ADK 记忆指南](https://google.github.io/adk-docs/sessions/memory/)** - 带代码示例的完整指南
- **[记忆工具](https://google.github.io/adk-docs/tools/built-in-tools/)** - PreloadMemory 和 LoadMemory 文档
- **[在 ADK 上使用 Memory Bank 入门](https://github.com/GoogleCloudPlatform/generative-ai/blob/main/agents/agent_engine/memory_bank/get_started_with_memory_bank_on_adk.ipynb)** - 演示如何构建带记忆的 ADK Agent 的示例 notebook

---
## 🧹 Section 6: 清理

### ⚠️ 重要：防止意外费用：测试完成后始终删除资源！

**成本提醒**

提醒一下，让 Agent 持续运行可能会产生费用。Agent Engine 提供每月免费层，你可以在[文档](https://docs.cloud.google.com/agent-builder/agent-engine/overview#pricing)中了解更多。

**测试完成后始终删除资源！**

当你完成测试和查询已部署的 Agent 后，建议删除远程 Agent 以避免产生额外费用：

In [ ]:
agent_engines.delete(resource_name=remote_agent.resource_name, force=True)

print("✅ Agent 删除成功")

**发生了什么：**

这个 cell 删除你已部署的 Agent：

- `resource_name=remote_agent.resource_name` - 标识要删除的 Agent
- `force=True` - 即使 Agent 正在运行也强制删除

删除过程通常需要 1-2 分钟。你可以在 [Agent Engine Console](https://console.cloud.google.com/vertex-ai/agents/agent-engines) 中验证删除。

---
## ✅ 恭喜！你已准备好生产部署

你已成功学会如何将 ADK Agent 部署到 Vertex AI Agent Engine - 将你的 Agent 从开发带到生产！

你现在知道如何用企业级基础设施部署 Agent、管理成本和测试生产部署。

### 📚 了解更多

参考以下文档了解更多：

- [ADK 文档](https://google.github.io/adk-docs/)
- [Agent Engine 文档](https://cloud.google.com/vertex-ai/generative-ai/docs/agent-engine/overview)
- [ADK 部署指南](https://google.github.io/adk-docs/deploy/agent-engine/)

**其他部署选项：**

- [Cloud Run 部署](https://google.github.io/adk-docs/deploy/cloud-run/)
- [GKE 部署](https://google.github.io/adk-docs/deploy/gke/)

**生产最佳实践：**

- 完成后删除测试部署以避免费用
- 启用追踪（`enable_tracing=True`）用于调试
- 通过 [Vertex AI Console](https://console.cloud.google.com/vertex-ai/agents/agent-engines) 监控
- 遵循[安全最佳实践](https://google.github.io/adk-docs/safety/)

---
## 🎯 课程回顾：你的 5 天学习之旅

在过去 5 天里，你学会了：

- **Day 1：** Agent 基础 - 用工具和指令构建你的第一个 Agent
- **Day 2：** 高级工具 - 自定义工具、内置工具和最佳实践
- **Day 3：** 会话和记忆 - 管理对话和长期知识存储
- **Day 4：** 可观测性和评估 - 监控 Agent 和衡量性能
- **Day 5：** 生产部署 - 用 Agent Engine 让你的 Agent 上线

你现在拥有构建、测试和部署生产就绪 AI Agent 的完整工具包！

### 🚀 下一步

**感谢你完成 5 天 AI Agent 课程！**

现在轮到你来构建：
- 开始用 ADK 创建你自己的 AI Agent
- 在 [Kaggle Discord](https://discord.com/invite/kaggle) 上与社区分享你的项目
- 在 [ADK 文档](https://google.github.io/adk-docs/) 中探索高级模式

**祝构建愉快！🚀**